# DMU0: IAC SDSS S82 extraction

The original extraction was missing objects below RA=0. In this notebook we combine the individual catalogues that were downloaded from ftp.iac.es and also available from http://www.iac.es/proyecto/stripe82/pages/data.php into one full catalogue to be used here and uploaded to CDS.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
017bb1e (Mon Jun 18 14:58:59 2018 +0100) [with local modifications]
This notebook was executed on: 
2018-09-21 17:03:17.431555


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table, vstack, join
import numpy as np
from pymoc import MOC

import glob

## I - Get list of files
Merge stars and galaxies into one table

In [3]:
file_list = glob.glob('./data/tiles/*.cat')
file_list[:10]

['./data/tiles/f0011_gal.cat',
 './data/tiles/f0011_ps.cat',
 './data/tiles/f0141_gal.cat']

In [4]:

test_gal = Table.read('./data/tiles/f0011_gal.cat', format='ascii')
test_star = Table.read('./data/tiles/f0011_ps.cat', format='ascii')

In [5]:
test_gal[:10]

col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,col71,col72,col73,col74,col75
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
310.089107,-1.2355282,22.7462,23.1497,24.375,23.3181,22.8383,22.858,0.3598,0.7628,0.3202,0.2261,0.2654,0.3903,4.348,20.6771,20.7329,22.4156,21.5,20.8421,20.7255,0.029,0.0418,0.0394,0.0281,0.025,0.0297,5.364,19.1425,19.1395,20.7003,19.8245,19.2839,19.1504,0.0109,0.014,0.0164,0.0114,0.01,0.0109,4.655,18.5251,18.5078,20.0178,19.1803,18.6769,18.531,0.01,0.0129,0.0137,0.0098,0.009,0.0099,4.514,18.2026,18.233,19.6292,18.7985,18.3173,18.21,0.0219,0.0303,0.0265,0.0193,0.0185,0.0217,4.232,3.5,5.28,5.192,3.678,-51.25,1.412,0.292,3
310.4277744,-1.240878,21.8127,21.7347,23.798,22.5158,21.8657,21.8246,0.1373,0.1644,0.209,0.1223,0.1167,0.1554,4.82,21.1949,21.1898,22.8877,21.7903,21.2244,21.1757,0.0386,0.0485,0.0522,0.0338,0.0328,0.0422,4.448,20.1127,20.1094,21.6674,20.654,20.1563,20.1062,0.0204,0.0251,0.0275,0.0184,0.0179,0.0223,4.207,19.749,19.7513,21.2348,20.262,19.79,19.7478,0.0231,0.0294,0.027,0.0189,0.0197,0.0257,4.072,19.6494,19.7808,20.9912,20.0667,19.6546,19.6937,0.062,0.0898,0.0588,0.0436,0.0501,0.0724,3.716,3.93,5.28,3.672,3.002,-13.12,1.223,0.182,0
310.3773283,-1.2421887,21.1581,20.6522,23.2656,22.2165,21.4613,21.0663,0.0833,0.0907,0.1464,0.0988,0.0847,0.0824,6.566,20.5964,20.7456,22.2701,21.3078,20.7641,20.5813,0.0246,0.0454,0.0359,0.0246,0.0231,0.0257,4.794,20.1955,20.2528,21.8629,20.9056,20.3452,20.1907,0.0222,0.0389,0.0307,0.0213,0.0203,0.0235,4.8,19.8286,19.7923,21.5589,20.5944,20.0465,19.7964,0.0253,0.0425,0.0328,0.0234,0.0238,0.0264,5.061,19.645,19.8544,21.5127,20.525,19.8841,19.6068,0.0632,0.136,0.0825,0.06,0.0594,0.0658,5.568,3.52,6.6,4.257,3.632,65.66,1.172,0.147,3
310.3473479,-1.2436515,20.9496,20.8676,23.1745,22.0697,21.3915,21.1149,0.0764,0.0886,0.1383,0.0895,0.0803,0.0856,7.436,19.5446,19.5233,21.3457,20.3725,19.8175,19.627,0.0124,0.0141,0.022,0.0145,0.0125,0.0129,5.391,18.541,18.5247,20.1045,19.2372,18.7668,18.6024,0.0073,0.0082,0.0121,0.0083,0.0073,0.0075,4.715,17.9975,17.9639,19.4694,18.6569,18.22,18.0607,0.0068,0.0077,0.0103,0.0073,0.0067,0.007,4.517,17.6516,17.6439,19.0806,18.2626,17.8474,17.7041,0.0142,0.0167,0.0198,0.0142,0.0133,0.0145,4.267,3.5,4.62,7.278,2.653,-43.49,2.743,0.635,0
310.2980108,-1.2450647,21.607,21.4455,22.5854,21.8443,21.6392,21.6538,0.1661,0.271,0.0972,0.0772,0.0971,0.1339,2.899,20.1728,20.0293,21.4897,20.7411,20.4794,20.3217,0.0237,0.0366,0.0237,0.0179,0.0193,0.0217,3.894,19.5099,19.4089,20.7507,20.113,19.8271,19.6646,0.0173,0.0278,0.0168,0.0134,0.0143,0.016,3.939,19.121,19.1886,20.4609,19.8242,19.4339,19.2395,0.0191,0.0366,0.0173,0.0142,0.015,0.0168,4.589,18.9867,19.4636,20.3576,19.7081,19.2606,19.1228,0.051,0.1467,0.0397,0.034,0.0367,0.0447,4.693,3.68,7.26,6.386,4.705,-45.82,1.357,0.263,3
310.1278317,-1.2453766,23.3106,23.2471,25.3441,23.6817,23.0434,22.7421,0.2596,0.3516,0.6795,0.2982,0.3114,0.3461,4.262,23.331,23.5734,24.214,23.4347,23.7548,24.6025,0.1259,0.2256,0.1305,0.1157,0.2815,0.9035,2.704,22.5988,22.5364,23.3221,22.6574,22.5755,22.8836,0.0846,0.1149,0.0791,0.0

In [6]:
test_star[:10]

col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,col71,col72,col73,col74,col75
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
310.0406973,-1.2388384,18.8416,18.8416,19.5305,18.9745,18.8274,18.7964,0.016,0.016,0.0209,0.0164,0.0161,0.0171,2.376,16.8871,16.8871,17.4646,16.9879,16.878,16.8573,0.0027,0.0027,0.0035,0.0028,0.0027,0.0027,2.145,16.1335,16.1335,16.6253,16.226,16.1239,16.101,0.0019,0.0019,0.0024,0.002,0.0019,0.0019,1.964,15.8372,15.8372,16.2734,15.9237,15.8279,15.8038,0.0019,0.0019,0.0023,0.0019,0.0019,0.0019,1.853,15.6703,15.6703,16.1325,15.7615,15.6608,15.6401,0.004,0.004,0.0048,0.0041,0.004,0.0042,1.919,3.5,3.5,2.574,2.509,54.91,1.026,0.025,2
310.4340371,-1.2408615,16.0954,16.0954,16.8229,16.234,16.0964,16.0726,0.0043,0.0043,0.006,0.0046,0.0043,0.0043,2.455,14.6855,14.6855,15.2957,14.7986,14.6862,14.6661,0.001,0.001,0.0013,0.001,0.001,0.001,2.223,14.2378,14.2378,14.7416,14.3367,14.2388,14.2174,0.0008,0.0008,0.001,0.0008,0.0008,0.0008,2.002,14.06,14.06,14.5014,14.1524,14.0609,14.0368,0.0008,0.0008,0.001,0.0009,0.0008,0.0008,1.88,13.9569,13.9569,14.4215,14.0521,13.9578,13.9351,0.0018,0.0018,0.0022,0.0019,0.0018,0.0018,1.931,3.5,3.5,3.002,2.79,53.86,1.076,0.071,3
310.1488541,-1.2407223,20.3273,20.3139,21.1015,20.5076,20.3018,20.2967,0.0358,0.0369,0.0444,0.0354,0.0377,0.0457,2.537,17.6022,17.5921,18.1504,17.694,17.5865,17.5671,0.0038,0.0038,0.0048,0.0039,0.0038,0.0039,2.086,16.2559,16.2454,16.7122,16.3355,16.2398,16.2184,0.002,0.002,0.0025,0.0021,0.002,0.002,1.898,15.7168,15.7059,16.1373,15.7948,15.7,15.6781,0.0018,0.0018,0.0021,0.0018,0.0018,0.0018,1.826,15.4062,15.3941,15.8584,15.4894,15.388,15.3682,0.0035,0.0035,0.0043,0.0036,0.0035,0.0036,1.903,3.5,3.96,2.36,2.331,-8.28,1.013,0.012,3
310.468454,-1.2408565,17.2796,17.2796,18.0317,17.4302,17.2847,17.2639,0.0075,0.0075,0.0104,0.0079,0.0075,0.0076,2.473,15.7684,15.7684,16.3965,15.8865,15.7719,15.7533,0.0016,0.0016,0.0021,0.0017,0.0016,0.0016,2.241,15.2211,15.2211,15.7054,15.3212,15.2252,15.2059,0.0012,0.0012,0.0015,0.0013,0.0012,0.0012,1.941,15.0532,15.0532,15.4835,15.1484,15.0572,15.0359,0.0013,0.0013,0.0016,0.0014,0.0013,0.0013,1.834,14.9819,14.9819,15.4594,15.0864,14.9852,14.9656,0.0029,0.0029,0.0035,0.003,0.0029,0.0029,1.943,3.5,3.5,3.674,2.598,60.38,1.414,0.293,2
310.3837857,-1.2413399,17.2569,17.2569,17.9518,17.3911,17.2494,17.2281,0.0074,0.0074,0.01,0.0078,0.0074,0.0075,2.402,15.922,15.922,16.5084,16.0296,15.9158,15.896,0.0017,0.0017,0.0022,0.0018,0.0017,0.0017,2.209,15.4462,15.4462,15.8933,15.5316,15.4401,15.4198,0.0014,0.0014,0.0017,0.0014,0.0014,0.0014,1.885,15.2785,15.2785,15.6734,15.3613,15.2719,15.249,0.0015,0.0015,0.0017,0.0015,0.0015,0.0015,1.762,15.2355,15.2355,15.6907,15.3286,15.2282,15.2069,0.0033,0.0033,0.0039,0.0034,0.0033,0.0033,1.933,3.5,3.5,2.645,2.591,-78.17,1.021,0.02,2
310.4004031,-1.2416517,20.2297,20.2297,20.9228,20.3601,20.1808,20.186,0.0339,0.0339,0.0408,0.0329,0.0352,0.0426,2.388,17.6512,17.6512,18.1989,17.7452,17.6334,17.6151,0.0039,0.0039,0.0049,0.004,0.0039,0.004,2.129,16.5732,16.5732,17.0009,16.6504,16.5566,16.5379,0.0023,0.0023,0.0028,0.0024

In [7]:
columns = {

    1:  'ALPHA_J2000'          ,#Right ascension of barycenter (J2000)                    [deg]
    2:  'DELTA_J2000'          ,#Declination of barycenter (J2000)                        [deg]
    3:  'MAG_AUTO_u'           ,#Sloan u | Kron-like elliptical aperture magnitude        [mag]
    4:  'MAG_PETRO_u'          ,#Sloan u | Petrosian-like elliptical aperture magnitude   [mag]
    5:  'MAG_APER_1_u'         ,#Sloan u | 5 pixel fixed aperture magnitude               [mag]
    6:  'MAG_APER_2_u'         ,#Sloan u | 10 pixel fixed aperture magnitude              [mag]
    7:  'MAG_APER_3_u'         ,#Sloan u | 20 pixel fixed aperture magnitude              [mag]
    8:  'MAG_APER_4_u'         ,#Sloan u | 30 pixel fixed aperture magnitude              [mag]
    9:  'MAGERR_AUTO_u'        ,#Sloan u | RMS error for AUTO magnitude                   [mag]
    10: 'MAGERR_PETRO_u'       ,#Sloan u | RMS error for PETROsian magnitude              [mag]
    11: 'MAGERR_APER_1_u'      ,#Sloan u | RMS error for MAG_APER (1)                     [mag]
    12: 'MAGERR_APER_2_u'      ,#Sloan u | RMS error for MAG_APER (2)                     [mag]
    13: 'MAGERR_APER_3_u'      ,#Sloan u | RMS error for MAG_APER (3)                     [mag]
    14: 'MAGERR_APER_4_u'      ,#Sloan u | RMS error for MAG_APER (4)                     [mag]
    15: 'FLUX_RADIUS_u'        ,#Sloan u | Fraction-of-light radii (0.5 of total)         [pixel] 
    16: 'MAG_AUTO_g'           ,#Sloan g | Kron-like elliptical aperture magnitude        [mag] 
    17: 'MAG_PETRO_g'          ,#Sloan g | Petrosian-like elliptical aperture magnitude   [mag] 
    18: 'MAG_APER_1_g'         ,#Sloan g | 5 pixel fixed aperture magnitude               [mag] 
    19: 'MAG_APER_2_g'         ,#Sloan g | 10 pixel fixed aperture magnitude              [mag] 
    20: 'MAG_APER_3_g'         ,#Sloan g | 20 pixel fixed aperture magnitude              [mag] 
    21: 'MAG_APER_4_g'         ,#Sloan g | 30 pixel fixed aperture magnitude              [mag] 
    22: 'MAGERR_AUTO_g'        ,#Sloan g | RMS error for AUTO magnitude                   [mag] 
    23: 'MAGERR_PETRO_g'       ,#Sloan g | RMS error for PETROsian magnitude              [mag] 
    24: 'MAGERR_APER_1_g'      ,#Sloan g | RMS error for MAG_APER (1)                     [mag] 
    25: 'MAGERR_APER_2_g'      ,#Sloan g | RMS error for MAG_APER (2)                     [mag] 
    26: 'MAGERR_APER_3_g'      ,#Sloan g | RMS error for MAG_APER (3)                     [mag] 
    27: 'MAGERR_APER_4_g'      ,#Sloan g | RMS error for MAG_APER (4)                     [mag] 
    28: 'FLUX_RADIUS_g'        ,#Sloan g | Fraction-of-light radii (0.5 of total)         [pixel] 
    29: 'MAG_AUTO_r'           ,#Sloan r | Kron-like elliptical aperture magnitude        [mag]  
    30: 'MAG_PETRO_r'          ,#Sloan r | Petrosian-like elliptical aperture magnitude   [mag]  
    31: 'MAG_APER_1_r'         ,#Sloan r | 5 pixel fixed aperture magnitude               [mag]  
    32: 'MAG_APER_2_r'         ,#Sloan r | 10 pixel fixed aperture magnitude              [mag]  
    33: 'MAG_APER_3_r'         ,#Sloan r | 20 pixel fixed aperture magnitude              [mag]  
    34: 'MAG_APER_4_r'         ,#Sloan r | 30 pixel fixed aperture magnitude              [mag]  
    35: 'MAGERR_AUTO_r'        ,#Sloan r | RMS error for AUTO magnitude                   [mag]  
    36: 'MAGERR_PETRO_r'       ,#Sloan r | RMS error for PETROsian magnitude              [mag]  
    37: 'MAGERR_APER_1_r'      ,#Sloan r | RMS error for MAG_APER (1)                     [mag]  
    38: 'MAGERR_APER_2_r'      ,#Sloan r | RMS error for MAG_APER (2)                     [mag]  
    39: 'MAGERR_APER_3_r'      ,#Sloan r | RMS error for MAG_APER (3)                     [mag]  
    40: 'MAGERR_APER_4_r'      ,#Sloan r | RMS error for MAG_APER (4)                     [mag]  
    41: 'FLUX_RADIUS_r'        ,#Sloan r | Fraction-of-light radii (0.5 of total)         [pixel]
    42: 'MAG_AUTO_i'           ,#Sloan i | Kron-like elliptical aperture magnitude        [mag]  
    43: 'MAG_PETRO_i'          ,#Sloan i | Petrosian-like elliptical aperture magnitude   [mag]  
    44: 'MAG_APER_1_i'         ,#Sloan i | 5 pixel fixed aperture magnitude               [mag]  
    45: 'MAG_APER_2_i'         ,#Sloan i | 10 pixel fixed aperture magnitude              [mag]  
    46: 'MAG_APER_3_i'         ,#Sloan i | 20 pixel fixed aperture magnitude              [mag]  
    47: 'MAG_APER_4_i'         ,#Sloan i | 30 pixel fixed aperture magnitude              [mag]  
    48: 'MAGERR_AUTO_i'        ,#Sloan i | RMS error for AUTO magnitude                   [mag]  
    49: 'MAGERR_PETRO_i'       ,#Sloan i | RMS error for PETROsian magnitude              [mag]  
    50: 'MAGERR_APER_1_i'      ,#Sloan i | RMS error for MAG_APER (1)                     [mag]  
    51: 'MAGERR_APER_2_i'      ,#Sloan i | RMS error for MAG_APER (2)                     [mag]  
    52: 'MAGERR_APER_3_i'      ,#Sloan i | RMS error for MAG_APER (3)                     [mag]  
    53: 'MAGERR_APER_4_i'      ,#Sloan i | RMS error for MAG_APER (4)                     [mag]  
    54: 'FLUX_RADIUS_i'        ,#Sloan i | Fraction-of-light radii (0.5 of total)         [pixel]
    55: 'MAG_AUTO_z'           ,#Sloan z | Kron-like elliptical aperture magnitude        [mag]  
    56: 'MAG_PETRO_z'          ,#Sloan z | Petrosian-like elliptical aperture magnitude   [mag]  
    57: 'MAG_APER_1_z'         ,#Sloan z | 5 pixel fixed aperture magnitude               [mag]  
    58: 'MAG_APER_2_z'         ,#Sloan z | 10 pixel fixed aperture magnitude              [mag]  
    59: 'MAG_APER_3_z'         ,#Sloan z | 20 pixel fixed aperture magnitude              [mag]  
    60: 'MAG_APER_4_z'         ,#Sloan z | 30 pixel fixed aperture magnitude              [mag]  
    61: 'MAGERR_AUTO_z'        ,#Sloan z | RMS error for AUTO magnitude                   [mag]  
    62: 'MAGERR_PETRO_z'       ,#Sloan z | RMS error for PETROsian magnitude              [mag]  
    63: 'MAGERR_APER_1_z'      ,#Sloan z | RMS error for MAG_APER (1)                     [mag]  
    64: 'MAGERR_APER_2_z'      ,#Sloan z | RMS error for MAG_APER (2)                     [mag]  
    65: 'MAGERR_APER_3_z'      ,#Sloan z | RMS error for MAG_APER (3)                     [mag]  
    66: 'MAGERR_APER_4_z'      ,#Sloan z | RMS error for MAG_APER (4)                     [mag]  
    67: 'FLUX_RADIUS_z'        ,#Sloan z | Fraction-of-light radii (0.5 of total)         [pixel]
    68: 'KRON_RADIUS'          ,#Kron aperture in units of A_IMAGE                        
    69: 'PETRO_RADIUS'         ,#Petrosian aperture in units of A_IMAGE                    
    70: 'A_IMAGE'              ,#Profile RMS along major axis                             [pixel]  
    71: 'B_IMAGE'              ,#Profile RMS along minor axis                             [pixel] 
    72: 'THETA_IMAGE'          ,#Position angle (CCW/x)                                   [deg] 
    73: 'ELONGATION'           ,#A_IMAGE/B_IMAGE
    74: 'ELLIPTICITY'          ,#1 - B_IMAGE/A_IMAGE    
    75: 'FLAGS'                ,#Extraction flags 
   
}

def fix_headers(cat):
    for col in columns:
        cat['col{}'.format(col)].name = columns[col]
        if columns[col].startswith('MAG'):
            cat[columns[col]].unit = u.mag
        if columns[col].startswith('FLUX_RADIUS_'):
            cat[columns[col]].unit = u.pix
    cat['A_IMAGE'].unit = u.pix
    cat['B_IMAGE'].unit = u.pix
    cat['THETA_IMAGE'].unit = u.deg
    cat['ALPHA_J2000'].unit = u.deg
    cat['DELTA_J2000'].unit = u.deg
    return cat

In [8]:
full_cat = Table()
for cat_name in file_list:
    try:
        cat = Table.read(cat_name, format='ascii')
    except: 
        print('Bad file: ' + cat_name)
        continue
    fix_headers(cat)
    
    cat.add_column(Column(data=np.zeros(len(cat)), name='sdss_stellarity', dtype='int32'))
    if 'ps.cat' in cat_name:
        cat['sdss_stellarity'] = 1
    
    full_cat = vstack([full_cat, cat])
    
    

Bad file: ./data/tiles/f0141_gal.cat


In [9]:
len(full_cat)

15694

In [10]:
full_cat.add_column(Column(data=np.arange(len(full_cat)), name='sdss_iac_id', dtype='int32'))

In [11]:
full_cat[full_cat.colnames]

ALPHA_J2000,DELTA_J2000,MAG_AUTO_u,MAG_PETRO_u,MAG_APER_1_u,MAG_APER_2_u,MAG_APER_3_u,MAG_APER_4_u,MAGERR_AUTO_u,MAGERR_PETRO_u,MAGERR_APER_1_u,MAGERR_APER_2_u,MAGERR_APER_3_u,MAGERR_APER_4_u,FLUX_RADIUS_u,MAG_AUTO_g,MAG_PETRO_g,MAG_APER_1_g,MAG_APER_2_g,MAG_APER_3_g,MAG_APER_4_g,MAGERR_AUTO_g,MAGERR_PETRO_g,MAGERR_APER_1_g,MAGERR_APER_2_g,MAGERR_APER_3_g,MAGERR_APER_4_g,FLUX_RADIUS_g,MAG_AUTO_r,MAG_PETRO_r,MAG_APER_1_r,MAG_APER_2_r,MAG_APER_3_r,MAG_APER_4_r,MAGERR_AUTO_r,MAGERR_PETRO_r,MAGERR_APER_1_r,MAGERR_APER_2_r,MAGERR_APER_3_r,MAGERR_APER_4_r,FLUX_RADIUS_r,MAG_AUTO_i,MAG_PETRO_i,MAG_APER_1_i,MAG_APER_2_i,MAG_APER_3_i,MAG_APER_4_i,MAGERR_AUTO_i,MAGERR_PETRO_i,MAGERR_APER_1_i,MAGERR_APER_2_i,MAGERR_APER_3_i,MAGERR_APER_4_i,FLUX_RADIUS_i,MAG_AUTO_z,MAG_PETRO_z,MAG_APER_1_z,MAG_APER_2_z,MAG_APER_3_z,MAG_APER_4_z,MAGERR_AUTO_z,MAGERR_PETRO_z,MAGERR_APER_1_z,MAGERR_APER_2_z,MAGERR_APER_3_z,MAGERR_APER_4_z,FLUX_RADIUS_z,KRON_RADIUS,PETRO_RADIUS,A_IMAGE,B_IMAGE,THETA_IMAGE,ELONGATION,ELLIPTICITY,FLAGS,sdss_stellarity,sdss_iac_id
deg,deg,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,pix,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,pix,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,pix,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,pix,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,pix,,,pix,pix,deg,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int32,int32
310.089107,-1.2355282,22.7462,23.1497,24.375,23.3181,22.8383,22.858,0.3598,0.7628,0.3202,0.2261,0.2654,0.3903,4.348,20.6771,20.7329,22.4156,21.5,20.8421,20.7255,0.029,0.0418,0.0394,0.0281,0.025,0.0297,5.364,19.1425,19.1395,20.7003,19.8245,19.2839,19.1504,0.0109,0.014,0.0164,0.0114,0.01,0.0109,4.655,18.5251,18.5078,20.0178,19.1803,18.6769,18.531,0.01,0.0129,0.0137,0.0098,0.009,0.0099,4.514,18.2026,18.233,19.6292,18.7985,18.3173,18.21,0.0219,0.0303,0.0265,0.0193,0.0185,0.0217,4.232,3.5,5.28,5.192,3.678,-51.25,1.412,0.292,3,0,0
310.4277744,-1.240878,21.8127,21.7347,23.798,22.5158,21.8657,21.8246,0.1373,0.1644,0.209,0.1223,0.1167,0.1554,4.82,21.1949,21.1898,22.8877,21.7903,21.2244,21.1757,0.0386,0.0485,0.0522,0.0338,0.0328,0.0422,4.448,20.1127,20.1094,21.6674,20.654,20.1563,20.1062,0.0204,0.0251,0.0275,0.0184,0.0179,0.0223,4.207,19.749,19.7513,21.2348,20.262,19.79,19.7478,0.0231,0.0294,0.027,0.0189,0.0197,0.0257,4.072,19.6494,19.7808,20.9912,20.0667,19.6546,19.6937,0.062,0.0898,0.0588,0.0436,0.0501,0.0724,3.716,3.93,5.28,3.672,3.002,-13.12,1.223,0.182,0,0,1
310.3773283,-1.2421887,21.1581,20.6522,23.2656,22.2165,21.4613,21.0663,0.0833,0.0907,0.1464,0.0988,0.0847,0.0824,6.566,20.5964,20.7456,22.2701,21.3078,20.7641,20.5813,0.0246,0.0454,0.0359,0.0246,0.0231,0.0257,4.794,20.1955,20.2528,21.8629,20.9056,20.3452,20.1907,0.0222,0.0389,0.0307,0.0213,0.0203,0.0235,4.8,19.8286,19.7923,21.5589,20.5944,20.0465,19.7964,0.0253,0.0425,0.0328,0.0234,0.0238,0.0264,5.061,19.645,19.8544,21.5127,20.525,19.8841,19.6068,0.0632,0.136,0.0825,0.06,0.0594,0.0658,5.568,3.52,6.6,4.257,3.632,65.66,1.172,0.147,3,0,2
310.3473479,-1.2436515,20.9496,20.8676,23.1745,22.0697,21.3915,21.1149,0.0764,0.0886,0.1383,0.0895,0.0803,0.0856,7.436,19.5446,19.5233,21.3457,20.3725,19.8175,19.627,0.0124,0.0141,0.022,0.0145,0.0125,0.0129,5.391,18.541,18.5247,20.1045,19.2372,18.7668,18.6024,0.0073,0.0082,0.0121,0.0083,0.0073,0.0075,4.715,17.9975,17.9639,19.4694,18.6569,18.22,18.0607,0.0068,0.0077,0.0103,0.0073,0.0067,0.007,4.517,17.6516,17.6439,19.0806,18.2626,17.8474,17.7041

In [12]:
cols = full_cat.colnames

cols.insert(0, cols.pop(cols.index('sdss_iac_id')))


In [13]:
full_cat[cols].write('./data/IAC-SDSS_Herschel-Stripe-82_full.fits', overwrite=True)